In [6]:
from google.colab import files
uploaded = files.upload()

Saving context.xlsx to context.xlsx


In [4]:
import pandas as pd

In [7]:
import io
df1 = pd.read_excel(io.BytesIO(uploaded['context.xlsx']))

In [5]:
import io
df2 = pd.read_excel(io.BytesIO(uploaded['questionAnswerData.xlsx']))

In [1]:
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [8]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla K80


In [9]:
!pip install transformers

     |████████████████████████████████| 1.8MB 5.8MB/s 
     |████████████████████████████████| 890kB 38.0MB/s 
     |████████████████████████████████| 3.2MB 39.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=001bc0827942037b521ca26f07c2ff16200ade72ded70bcf091512396fec2768
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [10]:
df1.head()

,Unnamed: 0,context,contextID
0,0,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,1
1,0,Following the disbandment of Destiny's Child i...,2
2,0,"A self-described ""modern-day feminist"", Beyonc...",3
3,0,"Beyoncé Giselle Knowles was born in Houston, T...",4
4,0,Beyoncé attended St. Mary's Elementary School ...,5


In [57]:

import ast

final_ans=df2['answers']

ans=[]
for i in range(0,len(final_ans)):
  if  final_ans[i]=="[]":
    ans.append("")
  else:
    ans.append(ast.literal_eval(final_ans[i][1:-1])['text'])

df2['ans']=ans


In [52]:

final_ans[1][1:-1]

"{'text': 'singing and dancing', 'answer_start': 207}"

In [60]:
cont=df1.context
ques=df2.question
answers=df2.ans

In [61]:
from transformers import BertTokenizer

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Loading BERT tokenizer...


In [64]:

def tokenize(data):
  input_ids = []

    # For every sentence...
  for sent in data:
    # `encode` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    encoded_sent = tokenizer.encode(
                    sent,                      # Sentence to encode.
                    add_special_tokens = True, # Add '[CLS]' and '[SEP]'

                    # This function also supports truncation and conversion
                    # to pytorch tensors, but we need to do padding, so we
                    # can't use these features :( .
                    #max_length = 128,          # Truncate all sentences.
                    #return_tensors = 'pt',     # Return pytorch tensors.
                )

    # Add the encoded sentence to the list.
    input_ids.append(encoded_sent)
    return input_ids

cont_id=tokenize(cont)
ques_id=tokenize(ques)

In [65]:
print('Max sentence length: ', max([len(sen) for sen in cont_id]))

Max sentence length:  166


In [66]:
 # We'll borrow the `pad_sequences` utility function to do this.
from keras.preprocessing.sequence import pad_sequences

def padding(input_ids):

    MAX_LEN = 166

    print('\nPadding/truncating all sentences to %d values...' % MAX_LEN)

    print('\nPadding token: "{:}", ID: {:}'.format(tokenizer.pad_token, tokenizer.pad_token_id))

    # Pad our input tokens with value 0.
    # "post" indicates that we want to pad and truncate at the end of the sequence,
    # as opposed to the beginning.
    input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", 
                            value=0, truncating="post", padding="post")
    print('\nDone.')
    return input_ids

cont_id=padding(cont_id)


Padding/truncating all sentences to 166 values...

Padding token: "[PAD]", ID: 0

Done.


Pre-trained bert

In [71]:
import torch
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer

#Model
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

In [96]:
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')


In [70]:
merged_left = pd.merge(left=df1, right=df2, how='left', left_on='contextID', right_on='contextID')
merged_left

,Unnamed: 0_x,context,contextID,Unnamed: 0_y,question,id,answers,is_impossible,plausible_answers,ans
0,0,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,1,0,When did Beyonce start becoming popular?,56be85543aeaaa14008c9063,"[{'text': 'in the late 1990s', 'answer_start':...",False,NaN,in the late 1990s
1,0,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,1,1,What areas did Beyonce compete in when she was...,56be85543aeaaa14008c9065,"[{'text': 'singing and dancing', 'answer_start...",False,NaN,singing and dancing
2,0,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,1,2,When did Beyonce leave Destiny's Child and bec...,56be85543aeaaa14008c9066,"[{'text': '2003', 'answer_start': 526}]",False,NaN,2003
3,0,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,1,3,In what city and state did Beyonce grow up?,56bf6b0f3aeaaa14008c9601,"[{'text': 'Houston, Texas', 'answer_start': 166}]",False,NaN,"Houston, Texas"
4,0,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,1,4,In which decade did Beyonce become famous?,56bf6b0f3aeaaa14008c9602,"[{'text': 'late 1990s', 'answer_start': 276}]",False,NaN,late 1990s
...,...,...,...,...,...,...,...,...,...,...
130314,0,"The term ""matter"" is used throughout physics i...",19035,0,Physics has broadly agreed on the definition o...,5a7e070b70df9f001a875439,[],True,"[{'text': 'matter', 'answer_start': 485}]",
130315,0,"The term ""matter"" is used throughout physics i...",19035,1,Who coined the term partonic matter?,5a7e070b70df9f001a87543a,[],True,"[{'text': 'Alfvén', 'answer_start': 327}]",
130316,0,"The term ""matter"" is used throughout physics i...",19035,2,What is another name for anti-matter?,5a7e070b70df9f001a87543b,[],True,"[{'text': 'Gk. common matter', 'answer_start':...",
130317,0,"The term ""matter"" is used throughout physics i...",19035,3,Matter usually does not need to be used in con...,5a7e070b70df9f001a87543c,[],True,"[{'text': 'a specifying modifier', 'answer_sta...",


In [73]:
question=merged_left.question
context=merged_left.context

In [81]:
# Apply the tokenizer to the input text, treating them as a text-pair.
input_ids = tokenizer.encode(question[0], context[0])

print('The input has a total of {:} tokens.'.format(len(input_ids)))

The input has a total of 174 tokens.


In [112]:
question[0]

'When did Beyonce start becoming popular?'

In [ ]:
# BERT only needs the token IDs, but for the purpose of inspecting the 
# tokenizer's behavior, let's also get the token strings and display them.
tokens = tokenizer.convert_ids_to_tokens(input_ids)

# For each token and its id...
for token, id in zip(tokens, input_ids):
    
    # If this is the [SEP] token, add some space around it to make it stand out.
    if id == tokenizer.sep_token_id:
        print('')
    
    # Print the token string and its ID in two columns.
    print('{:<12} {:>6,}'.format(token, id))

    if id == tokenizer.sep_token_id:
        print('')
    

In [92]:
# Search the input_ids for the first instance of the `[SEP]` token.
sep_index = input_ids.index(tokenizer.sep_token_id)

# The number of segment A tokens includes the [SEP] token istelf.
num_seg_a = sep_index + 1

# The remainder are segment B.
num_seg_b = len(input_ids) - num_seg_a

# Construct the list of 0s and 1s.
segment_ids = [0]*num_seg_a + [1]*num_seg_b

# There should be a segment_id for every input token.
assert len(segment_ids) == len(input_ids)

In [117]:
# Run our example through the model.
start_scores, end_scores = model(torch.tensor([input_ids]), token_type_ids=torch.tensor([segment_ids]),return_dict=False)  

In [120]:
input_ids

[101,
 2043,
 2106,
 20773,
 2707,
 3352,
 2759,
 1029,
 102,
 20773,
 21025,
 19358,
 22815,
 1011,
 5708,
 1006,
 1013,
 12170,
 23432,
 29715,
 3501,
 29678,
 12325,
 29685,
 1013,
 10506,
 1011,
 10930,
 2078,
 1011,
 2360,
 1007,
 1006,
 2141,
 2244,
 1018,
 1010,
 3261,
 1007,
 2003,
 2019,
 2137,
 3220,
 1010,
 6009,
 1010,
 2501,
 3135,
 1998,
 3883,
 1012,
 2141,
 1998,
 2992,
 1999,
 5395,
 1010,
 3146,
 1010,
 2016,
 2864,
 1999,
 2536,
 4823,
 1998,
 5613,
 6479,
 2004,
 1037,
 2775,
 1010,
 1998,
 3123,
 2000,
 4476,
 1999,
 1996,
 2397,
 4134,
 2004,
 2599,
 3220,
 1997,
 1054,
 1004,
 1038,
 2611,
 1011,
 2177,
 10461,
 1005,
 1055,
 2775,
 1012,
 3266,
 2011,
 2014,
 2269,
 1010,
 25436,
 22815,
 1010,
 1996,
 2177,
 2150,
 2028,
 1997,
 1996,
 2088,
 1005,
 1055,
 2190,
 1011,
 4855,
 2611,
 2967,
 1997,
 2035,
 2051,
 1012,
 2037,
 14221,
 2387,
 1996,
 2713,
 1997,
 20773,
 1005,
 1055,
 2834,
 2201,
 1010,
 20754,
 1999,
 2293,
 1006,
 2494,
 1007,
 1010,
 2029,
 25

In [118]:
# Find the tokens with the highest `start` and `end` scores.
answer_start = torch.argmax(start_scores)
answer_end = torch.argmax(end_scores)

# Combine the tokens in the answer and print it out.
answer = ' '.join(tokens[answer_start:answer_end+1])

print('Answer: "' + answer + '"')

Answer: "late 1990s"


In [119]:
# Start with the first token.
answer = tokens[answer_start]

# Select the remaining answer tokens and join them with whitespace.
for i in range(answer_start + 1, answer_end + 1):
    
    # If it's a subword token, then recombine it with the previous token.
    if tokens[i][0:2] == '##':
        answer += tokens[i][2:]
    
    # Otherwise, add a space then the token.
    else:
        answer += ' ' + tokens[i]

print('Answer: "' + answer + '"')

Answer: "late 1990s"


In [ ]:
def pre-process_inputID(question, answer_text):
    '''
    Takes a `question` string and an `answer_text` string (which contains the
    answer), and identifies the words within the `answer_text` that are the
    answer. Prints them out.
    '''
    # ======== Tokenize ========
    # Apply the tokenizer to the input text, treating them as a text-pair.
    input_ids = tokenizer.encode(question, answer_text)
    return input_ids

def sep_id(input_ids)
    # ======== Set Segment IDs ========
    # Search the input_ids for the first instance of the `[SEP]` token.
    sep_index = input_ids.index(tokenizer.sep_token_id)

    # The number of segment A tokens includes the [SEP] token istelf.
    num_seg_a = sep_index + 1

    # The remainder are segment B.
    num_seg_b = len(input_ids) - num_seg_a

    # Construct the list of 0s and 1s.
    segment_ids = [0]*num_seg_a + [1]*num_seg_b

    # There should be a segment_id for every input token.
    assert len(segment_ids) == len(input_ids)
    return segment_ids

In [ ]:
all_input_ids=[]
